# Prompting Langchain

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*DkgQ23ng-8J2sl66_YQWnw.png">

In [2]:
!python -m pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [13]:
!pip install langchain-openai
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install langchain-unstructured
!pip install gradio
!pip install unstructured
!pip install "unstructured[all-docs]"
!pip install faiss-gpu
!pip install Chromadb
!pip install numpy
!pip install langdetect
!pip install SqlAlchemy

  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl (316 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.27.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.28.2 which is incompatible.
  Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl (

In [3]:
import os

# from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_unstructured import UnstructuredLoader
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
# from langchain_unstructured import UnstructuredLoader
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from pydantic import BaseModel, Field
from typing import List
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from operator import itemgetter
from langdetect import detect, detect_langs

In [4]:
# file = 'CRA023005V5-ASS28082024V01-000727267.pdf'
file = '../files/118498_ocr.pdf'
# file = '573735.pdf'
# file = "726644.pdf" # Assembleia

encoding_name="cl100k_base"
path_faiss="/tmp/faiss_index"
deployment_name = "leitura_documento_gpt4o"
embedding_model = "text-embedding"
openai_api_version="2023-05-15"
chunk_size = 2500
chunk_overlap = 512

model = "gpt-4o"

verbose=False
score_threshold = 0.5
temperature = 0.0
k = 8
fetch_k = 30

In [22]:
!pip install pytesseract pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 7.5 MB/s eta 0:00:0000:0100:01


In [5]:
def raw_txt_from_txtpdf(in_file): 
    with pymupdf.open(in_file) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
 
    return text

In [6]:
raw_txt = ""
raw_doc = ""

In [7]:
import pymupdf
import io
from PIL import Image
import pytesseract

def raw_txt_from_imgpdf(in_file):
    docs = []
    doc = pymupdf.open(in_file) # open document
    full_text = ""
    for page in doc: # iterate through the pages
        pix = page.get_pixmap(dpi=300) # render page to an image
        pix_bytes = pix.tobytes()
 
        img = Image.open(io.BytesIO(pix_bytes))
 
        text = pytesseract.image_to_string(img, lang="por")
        docs.append(text)
 
    return docs

docs = raw_txt_from_imgpdf(file)
print(f"Pages: {len(docs)}")

raw_txt = "\n\n".join(doc for doc in docs)
raw_txt

Pages: 90


'Autorregulação\n\nANBIMA\n\nAgente Fiduciário\n\n \n\nTERMO DE SECURITIZAÇÃO DE CRÉDITOS IMOBILIÁRIOS\n\nCERTIFICADOS DE RECEBÍVEIS IMOBILIÁRIOS\nDA 2392 SÉRIE DA 12 EMISSÃO DA\n\nm\nM RBSEC\n\nRB CAPITAL COMPANHIA DE SECURITIZAÇÃO\nCompanhia Aberta\nCNPJ nº 02.773.542/0001-22\n\n16 de dezembro de 2019.\n\nJUR SP - 35582069v3 - 12767002.447539\n\x0c\n\n \n\nAutorregulação\n\n \n\nANBIMA\nAgente Fiduciário\nÍNDICE\n[OD] =| (O (6 | = PR 3\nII OBJETO E CRÉDITOS IMOBILIÁRIOS ....eteteeeeereresasaaaenanaeeeereeessssasaaaaaanaaetess 14\nHI IDENTIFICAÇÃO DOS CRI E FORMA DE DISTRIBUIÇÃO .........eeeeeeeeeeeeeeeeeeeseesss 15\nIV SUBSCRIÇÃO E INTEGRALIZAÇÃO DOS CRI ........ccteetterasereraereeaaserenanereeanaoa 18\nV CÁLCULO DO SALDO DEVEDOR COM ATUALIZAÇÃO MONETÁRIA, JUROS\nREMUNERATÓRIOS E AMORTIZAÇÃO.........ttttteeeaaaaarererereeaaaaaaaaaeerrraaaeaaaaaaaaaas 19\nVI AMORTIZAÇÃO EXTRAORDINÁRIA E RESGATE ANTECIPADO DOS CRI ........ees 23\nVII OBRIGAÇÕES E DECLARAÇÕES DA EMISSORA......ttemeeeme

# Loading doc using Unstructured-IO

In [17]:
print("Loading Unstructured Data")
# modes: “single”, “elements”, and “paged”
# loader = UnstructuredFileLoader(pdf, mode="elements")
loader = UnstructuredLoader(file, mode="paged", strategy="hi_res", chunking_strategy="by_title", max_characters=chunk_size, new_after_n_chars=chunk_size, )
raw_doc = loader.load()
raw_doc

Loading Unstructured Data


INFO: Reading PDF for file: ../files/118498.pdf ...


NameError: name 'sort_page_elements' is not defined

In [24]:
# str_template_prompt_initial = \
# """
# Responda à pergunta o mais detalhadamente possível a partir do contexto fornecido.
# Certifique-se de fornecer todos os detalhes. Quebre as tarefas em pequenas tarefas e passos.
# Caso a resposta não estiver no contexto fornecido, basta dizer "a resposta não está disponível no contexto".
# Não forneça respostas erradas ou respostas que não estão no contexto fornecido.
# Não coloque comentários dentro do JSON retornado para explicar os atributos.
# 
# QUESTION: {question}
# =========
# {summaries}
# =========
# FINAL ANSWER:
# SOURCES:
# """

# Splitting doc in chuncks using RecursiveCharacterTextSplitter

In [8]:
# segmenting the document into segments
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", " ", ""])
if raw_txt:
    chunk_txt = text_splitter.split_text(raw_txt)
else:    
    chunk_docs = text_splitter.split_documents(raw_doc)

In [9]:
if raw_txt:
    print(chunk_txt[0])
else:
    print(chunk_docs[0])

Autorregulação

ANBIMA

Agente Fiduciário

 

TERMO DE SECURITIZAÇÃO DE CRÉDITOS IMOBILIÁRIOS

CERTIFICADOS DE RECEBÍVEIS IMOBILIÁRIOS
DA 2392 SÉRIE DA 12 EMISSÃO DA

m
M RBSEC

RB CAPITAL COMPANHIA DE SECURITIZAÇÃO
Companhia Aberta
CNPJ nº 02.773.542/0001-22

16 de dezembro de 2019.

JUR SP - 35582069v3 - 12767002.447539


 

Autorregulação

 

ANBIMA
Agente Fiduciário
ÍNDICE
[OD] =| (O (6 | = PR 3
II OBJETO E CRÉDITOS IMOBILIÁRIOS ....eteteeeeereresasaaaenanaeeeereeessssasaaaaaanaaetess 14
HI IDENTIFICAÇÃO DOS CRI E FORMA DE DISTRIBUIÇÃO .........eeeeeeeeeeeeeeeeeeeseesss 15
IV SUBSCRIÇÃO E INTEGRALIZAÇÃO DOS CRI ........ccteetterasereraereeaaserenanereeanaoa 18
V CÁLCULO DO SALDO DEVEDOR COM ATUALIZAÇÃO MONETÁRIA, JUROS
REMUNERATÓRIOS E AMORTIZAÇÃO.........ttttteeeaaaaarererereeaaaaaaaaaeerrraaaeaaaaaaaaaas 19
VI AMORTIZAÇÃO EXTRAORDINÁRIA E RESGATE ANTECIPADO DOS CRI ........ees 23
VII OBRIGAÇÕES E DECLARAÇÕES DA EMISSORA......ttemeeememeeeeeeenenererenereeatens 27
VIII GARANTIAS.

# Get Embeddings

In [10]:
embeddings = AzureOpenAIEmbeddings(model=embedding_model)

# Saving chunks in local database

In [11]:
if raw_txt:
    metadatas = [{"source": file, "filename": file} for d in chunk_txt]
    vector_store = FAISS.from_texts(chunk_txt, embedding=embeddings, metadatas=metadatas)
else:
    vector_store = FAISS.from_documents(chunk_docs, embedding=embeddings)
vector_store.save_local(path_faiss)

# Creating Prompt TS - Escritura

In [26]:
from langchain_core.output_parsers import JsonOutputParser
 
class SecuritizadoraModel(BaseModel):
    """Parte nomeadas e qualificadas como Securitizadora ou Emissora"""
    cnpj: str = Field(description="Número do CNPJ da Securitizadora ou Emissora")
    nome: str = Field(description="Nome da empresa ou Razão Social da Securitizadora ou Emissora")

class AgenteFiduciarioModel(BaseModel):
    """Parte nomeadas e qualificadas como Agente Fiduciário"""
    cnpj: str = Field(description="Número do CNPJ do Agente Fiduciário")
    nome: str = Field(description="Nome da empresa ou Razão Social do Agente Fiduciário")

SecuritizadoraModel.model_rebuild()
AgenteFiduciarioModel.model_rebuild()

# Define your desired data structure.
class TermoSecModel(BaseModel):
    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA; 
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI; 
    Sendo DEBENTURES = DEB; Sendo CERTIFICADOS DE RECEBIVEIS = CR; 
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)
    numero_emissao: str = Field(description="""
    Número da emissão do Certificado de Recebíveis em sua forma apenas numérica.
    """)
    numero_serie: List[str] = Field(description="""
    Número da série em sua forma apenas numérica. 
    Caso seja uma série única, retorne UNI.
    """)
    securitizadora_emissora: SecuritizadoraModel = Field(description="Emissora ou Securitizadora")
    
    agente_fiduciario: AgenteFiduciarioModel = Field(description="Agente Fiduciário")

TermoSecModel.model_rebuild()

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=TermoSecModel)

# Creating Prompt Assembleia

In [144]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from datetime import date

class AssembleiaModel(BaseModel):    
    tipo: str = Field(description="""
    Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA; 
    Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI; 
    Sendo DEBENTURES = DEB; Sendo CERTIFICADOS DE RECEBIVEIS = CR; 
    Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''.
    """)
    numero_emissao: str = Field(description="""
    Número da emissão em sua forma apenas numérica.
    """)
    numero_serie: List[str] = Field(description="""
    Número da série em sua forma apenas numérica. 
    Caso seja uma série única, retorne UNI.
    """)
    dt_assembleia: date = Field(description="A data da assembleia no documento")
    hora_assembleia: str = Field(description="A hora da assembleia no documento")
    nr_convocacao: int = Field(description="O número de convocação da assembleia")
    fl_assembleia_deliberada: bool = Field(description="A assembleia teve a Ordem do Dia deliberada?")
    fl_assembleia_nao_instalada: bool = Field(description="A assembleia não foi instalada?")
    categorias: List[str] = Field(description="Quais as categorias do texto")
    
parser_assembleia = JsonOutputParser(pydantic_object=AssembleiaModel)

# Get Retriever

In [13]:
filename_list = [file]
filename_filter = {"filename": file}

score_threshold = 0.6
k = 5
fetch_k = 20

retriever = vector_store.as_retriever(
    # search_type="similarity_score_threshold",
    search_type="mmr",
    search_kwargs={
        # "score_threshold": score_threshold, 
        "k": k, "fetch_k": fetch_k, 
        "filter": filename_filter,
        "lambda_mult": 0.25,
    }
)

retriever

VectorStoreRetriever(tags=['FAISS', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ecedf4cc3d0>, search_type='mmr', search_kwargs={'k': 5, 'fetch_k': 20, 'filter': {'filename': '../files/118498_ocr.pdf'}, 'lambda_mult': 0.25})

# Question to LLM

In [16]:
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = ("""
Você é um assistente de IA que extrai dados de documentos e os retorna como objetos JSON estruturados. Não retorne como um bloco de código.

"{context}"
"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

llm = AzureChatOpenAI(azure_deployment=deployment_name, openai_api_version=openai_api_version,)

# question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Executing LLM - TS - Escritura

In [21]:
from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers.document_compressors import LLMChainExtractor

campos = [
    'Tipo do Certificado', 
    'Número da emissão da Securitizadora ou Emissora', 
    'Número da série da emissão da Securitizadora ou Emissora', 
    'Emissora ou Securitizadora', 'Número do CNPJ da Emissora ou Securitizadora', 
    'Agente Fiduciário', 'Número do CNPJ do Agente Fiduciário',
]


input_prompt = PromptTemplate.from_template("""
Extraia os dados deste contrato usando somente a estrutura JSON fornecida. Forneça somente valores para os campos na estrutura. Se um valor não estiver presente, forneça nulo. Valores na estrutura podem ser inferidos com base em outros valores e regras definidos no texto.
{nomes_campos}
""", partial_variables={'nomes_campos': ', '.join(campos)})

# FlashrankRerank.update_forward_refs()
# compressor = FlashrankRerank()
# chain_retrieve = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

# compressor = LLMChainExtractor.from_llm(llm)
# chain_retrieve = ContextualCompressionRetriever(
#     base_compressor=compressor, base_retriever=retriever
# )
# chain_retrieve = retriever
# docs = chain_retrieve.invoke(input_prompt.format())

# print(docs)

def format_docs(docs):
    return "\n\n ------------".join(doc.page_content for doc in docs)

input_prompt.format()

'\nExtraia os dados deste contrato usando somente a estrutura JSON fornecida. Forneça somente valores para os campos na estrutura. Se um valor não estiver presente, forneça nulo. Valores na estrutura podem ser inferidos com base em outros valores e regras definidos no texto.\nTipo do Certificado, Número da emissão da Securitizadora, Número da série da emissão da Securitizadora, Emissora ou Securitizadora, Número do CNPJ da Emissora ou Securitizadora, Agente Fiduciário, Número do CNPJ do Agente Fiduciário\n'

In [29]:
llm_with_schema = llm.with_structured_output(TermoSecModel)

rag_chain = (RunnableParallel(context = retriever | format_docs, input = RunnablePassthrough())
            | prompt
            | llm_with_schema)

result = rag_chain.invoke("nExtraia os dados deste contrato")

result

TermoSecModel(tipo='CRI', numero_emissao='1', numero_serie=['2392'], securitizadora_emissora=SecuritizadoraModel(cnpj='89.960.090/0001-76', nome='RB Capital Companhia de Securitização'), agente_fiduciario=AgenteFiduciarioModel(cnpj='', nome=''))

# Executing LLM - Assembleia

In [145]:
input_prompt = PromptTemplate.from_template("""
Extraia as seguintes informações:
{nomes_campos}
- Tipo da Emissão
- Número da Emissão da Securitizadora
- Séries da Emissão
- Data da realização da Assembleia
- Hora da realização da Assembleia
- Número de convocação da Assembleia
- A Assembleia foi deliberada?
- A Assembleia não foi instalada?
- Quais as categorias do texto? Múltiplas Escolhas:
    - Alteração Fluxos ou Datas de Pagamentos
    - Alteração Índices Financeiros
    - Alteração Índices/Razão de Garantia ou LTV
    - Alteração Vencimento Final da Operação
    - Aprovação das Demonstrações Financeiras
    - Aprovação de Venda de Imóvel
    - Contratação de Assessor Legal
    - Contração de Prestador de Serviços
    - Descumprimento Prazo de Registro de Garantia
    - Descumprimento Demais Obrigações Não Pecuniárias
    - Descumprimento Obrigações Pecuniárias
    - Liberação de Recursos de Conta
    - Realização de Aporte pelos Investidores
    - Reorganização Societária
    - Substituição/Inclusão/Liberação de Garantia
    - Utilização Diversa ou Alteração da Cascata
    - Dispensa ou Alteração dos Mecanismos de Resgate/Recompra
    - Emissão de Nova Série
    - Reestruturação
    - Medidas Iniciais Pós Vencimento
    - Alteração ou Recomposição de Fundos
    - Aprovação de Medidas Judiciais e Extrajudiciais
    - Outros

{format_instructions}
""", partial_variables={'nomes_campos': ', '.join(campos), 'format_instructions': parser.get_format_instructions()})

chain_retrieve = retriever
docs = chain_retrieve.invoke(input_prompt.format())
print(docs)

docs_formatted = "\n\n".join(doc.page_content for doc in docs)
# docs_formatted

[Document(page_content='DocuSign Envelope ID: FEC536E9-518F-4101-8EE7-7A87E342D750\n\nTRUE\n\nE SECURITEADORA\n\nTRUE SECURITIZADORA S.A.\nCNPJ Nº 12.130.744/0001-00\nNIRE 35.300.444.957\n\nATA DA ASSEMBLEIA ESPECIAL DOS TITULARES DE CERTIFICADOS DE\nRECEBÍVEIS IMOBILIÁRIOS DAS 1", 2º E 3º SÉRIES DA 17º EMISSÃO DA TRUE\nSECURITIZADORA S.A.\n\nREALIZADA EM 26 DE AGOSTO DE 2024\n\n1. Data, Hora e Local da Assembleia: Realizada às 9h do dia 26 de agosto de 2024, de forma\nremota e exclusivamente de modo digital, nos termos da Resolução da Comissão de Valores Mobiliários\n(CCVM?) nº 60, de 23 de dezembro de 2021 (“Resolução CVM 60º), com a dispensa de\nvideoconferência em razão da presença dos Titulares dos CRI (conforme definido abaixo),\nrepresentando 100% (cem por cento) dos CRI em Circulação (conforme definido no Termo de\nSecuritização), sendo considerada para todos os fins legais coordenada pela True Securitizadora S.A.\n(“Emissora”), situada na cidade de São Paulo, estado de São Pau

In [146]:
llm_with_schema = llm.with_structured_output(AssembleiaModel)

chain = prompt | llm_with_schema

result = chain.invoke({"context": docs_formatted, "input": input_prompt.format()})
result

AssembleiaModel(tipo='CRI', numero_emissao='17', numero_serie=['1', '2', '3'], dt_assembleia=datetime.date(2024, 8, 26), hora_assembleia='09:00', nr_convocacao=1, fl_assembleia_deliberada=True, fl_assembleia_nao_instalada=False, categorias=['Alteração Fluxos ou Datas de Pagamentos', 'Descumprimento Obrigações Pecuniárias'])

In [15]:
# campos = [
#     ("tipo", "str", "Sendo CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO = CRA; Sendo CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS = CRI; Sendo DEBENTURES = DEB; Sendo CERTIFICADOS DE RECEBIVEIS = CR; Retorne se CRI, CRA, CR ou DEB. Caso não encontre, retorne ''."),
#     ("numero_emissao", "str", "Número da emissão da Securitizadora. Retorne apenas a forma numérica."),
#     ("numero_serie", "str", "Número da série da emissão. Retorne apenas a forma numérica. Caso seja uma série única, retorne UNI. Caso encontre mais de uma, retorne separados por ','."),
#     ("nome_securitizadora_emissora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Emissora ou Securitizadora. Caso não encontre retorne ''"),
#     ("nome_agente_fiduciario", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Agente Fiduciário. Caso não encontre retorne ''"),
#     ("nome_devedora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Devedora ou que deve. Caso não encontre retorne ''. Caso encontre mais de um, retorne separados por ','"),
#     ("nome_avalista_fiadora", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Avalista ou Fiador. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''"),
#     ("numero_avalista_fiadora", "str", "Número de inscrição no CNPJ do Avalista ou Fiador {nome_avalista_fiadora}. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
#     ("numero_cnpj_securitizadora_emissora", "str", "Número da inscrição no Cadastro Nacional de Pessoa Jurídica (“CNPJ”) da Emissora ou Securitizadora - {nome_securitizadora_emissora}. Caso não encontre retorne ''"),
#     ("numero_cnpj_agente_fiduciario", "str", "Número da inscrição no Cadastro Nacional de Pessoa Jurídica (“CNPJ”) do Agente Fiduciário - {nome_agente_fiduciario}. Caso não encontre retorne ''"),
#     ("numero_cnpj_devedora", "str", "Número de inscrição no CNPJ da Devedora {nome_devedora}. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
#     ("nome_cedente", "str", "Nome ou empresa ou sociedade cuja denominação ou significado está como Cedente(s) ou que cedeu. Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''."),
#     ("numero_cpf_cnpj_cedente", "str", "Número de inscrição no CNPJ ou CPF do(s) Cedente(s). Caso encontre mais de um, retorne separados por ','. Caso não encontre retorne ''"),
#     ("numero_minimo_instalacao_assembleia_primeira_convocacao", "str", "Número mínimo de presentes para instalação de assembleia em primeira convocação. Caso não encontre retorne ''. Caso encontre, retorne o mais simples possível."),
#     ("numero_minimo_instalacao_assembleia_segunda_convocacao", "str", "Número mínimo de presentes para instalação de assembleia em segunda convocação. Caso não encontre retorne ''. Caso encontre, retorne o mais simples possível.")
# ]